In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.6931, 0.1476, 0.1960],
        [0.9572, 0.0783, 0.6816],
        [0.9648, 0.8105, 0.5239],
        [0.6730, 0.5455, 0.4506],
        [0.5758, 0.6720, 0.2585]])


In [3]:
import torch
torch.cuda.is_available()

False

In [4]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [5]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [6]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [9]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [10]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
tensor([[-0.0706, -0.0615, -0.0502,  0.0008, -0.0235, -0.0705, -0.0473, -0.0443,
          0.0382, -0.0224],
        [-0.0532, -0.0850, -0.0412, -0.0248, -0.0559,  0.0201, -0.0593, -0.0343,
          0.0826, -0.0057],
        [-0.0309, -0.0555,  0.0070, -0.0306, -0.0077,  0.0190, -0.0400, -0.0008,
          0.0645,  0.0024],
        [-0.0506, -0.0684, -0.0192, -0.0069, -0.0135,  0.0181, -0.0254, -0.0090,
          0.0855, -0.0043],
        [-0.0384, -0.1089, -0.0475, -0.0101, -0.0202,  0.0214, -0.0782,  0.0015,
          0.0945, -0.0170],
        [-0.0862, -0.0670, -0.0536, -0.0373, -0.0111,  0.0043, -0.0509, -0.0531,
          0.0887, -0.0274],
        [-0.0158, -0.0785, -0.0097, -0.0283, -0.0071,  0.0094,  0.0198,  0.0041,
          0.0941, -0.0271],
        [-0.1230, -0.0645, -0.0850, -0.0319, -0.0041,  0.0194, -0.0426, -0.0644,
          0.1131, -0.0147],
        [-0.0171, -0.0473,  0.0027, -0.0424,  0.0118,  0.0158, -0.0281, -0.0337,
       

tensor([[-0.0042, -0.0463,  0.0240, -0.0481,  0.0070,  0.0097, -0.0033, -0.0018,
          0.0451, -0.0503],
        [-0.0435, -0.0609,  0.0083, -0.0580, -0.0070, -0.0206, -0.0279, -0.0422,
          0.0512, -0.0398],
        [-0.0225, -0.0826, -0.0149, -0.0213, -0.0025, -0.0357, -0.0672, -0.0338,
          0.0564, -0.0138],
        [-0.0365, -0.0579,  0.0151, -0.0353,  0.0236, -0.0346, -0.0049, -0.0447,
          0.0625, -0.0232],
        [-0.0719, -0.0857, -0.0318, -0.0595,  0.0268, -0.0056, -0.0080, -0.0248,
          0.1209, -0.0178],
        [-0.0092, -0.0882, -0.0023, -0.0336,  0.0202, -0.0026, -0.0200,  0.0163,
          0.0814,  0.0183],
        [-0.0697, -0.0958, -0.0232,  0.0094,  0.0241, -0.1044, -0.0617, -0.0806,
          0.0402,  0.0394],
        [-0.0899, -0.0727, -0.0399, -0.0783,  0.0291, -0.0532, -0.0030, -0.0274,
          0.1004, -0.0027],
        [-0.0433, -0.0757, -0.0019, -0.0697,  0.0229, -0.0279, -0.0022, -0.0617,
          0.0937, -0.0337],
        [-0.0105, -

tensor([[-6.7164e-02, -8.0041e-02, -2.3350e-02, -5.6985e-02,  1.2827e-02,
         -8.9423e-02, -4.1963e-02, -5.9741e-02,  6.9930e-02,  6.3737e-02],
        [-4.0196e-03, -9.3299e-02,  1.9703e-02, -3.0717e-02,  6.7947e-02,
         -1.1349e-01, -2.1273e-02, -1.2251e-01,  6.3727e-02, -3.6366e-02],
        [ 5.0999e-02, -9.4945e-02,  1.4682e-02, -9.1865e-03,  2.9278e-02,
         -1.5827e-01, -6.3574e-02, -1.2907e-01,  1.5031e-02, -3.3542e-02],
        [-3.4374e-02, -6.3139e-02, -1.8655e-03, -5.4932e-02, -5.2310e-03,
         -1.5299e-02, -3.9445e-03,  2.8027e-02,  8.2846e-02, -6.3588e-03],
        [-4.4670e-03, -7.9982e-02,  4.0320e-02, -3.6982e-02,  8.9553e-02,
         -8.5667e-02,  7.5714e-03, -6.8692e-02,  5.7399e-02,  2.4187e-02],
        [ 2.3629e-02, -6.7671e-02,  2.4373e-02,  3.8593e-02,  6.1063e-02,
         -1.2552e-01, -5.9131e-02, -9.0713e-02,  4.9027e-02, -6.0626e-02],
        [ 2.5644e-02, -3.5141e-02, -9.4957e-03,  3.7131e-02, -9.5104e-03,
         -7.2848e-02, -6.3989e-0

tensor([[ 1.2299e-02, -1.6521e-02, -1.0059e-02,  3.5104e-02,  2.4587e-02,
         -6.9084e-02, -6.7141e-02, -6.2930e-02,  2.6072e-02, -6.2360e-02],
        [-6.4502e-02, -9.6949e-02,  4.5326e-02, -7.8073e-02,  8.6463e-02,
         -6.1354e-02,  1.5081e-02, -8.1866e-02,  7.1890e-02, -1.3203e-02],
        [-5.3219e-02, -1.1855e-01,  5.8436e-02, -5.6386e-02,  9.4884e-02,
         -1.1783e-01,  2.7142e-02, -1.2466e-01,  7.8377e-02, -6.2592e-02],
        [ 7.0317e-02, -8.0864e-02,  2.5846e-02, -1.4291e-02,  5.9408e-02,
         -1.4219e-01, -2.2323e-02, -1.1228e-01,  4.2263e-03, -4.6167e-02],
        [-7.2603e-02, -1.0679e-01, -2.9676e-03, -8.8401e-02,  3.4162e-02,
         -6.7713e-02, -3.2305e-02,  1.4399e-02,  8.3567e-02,  1.0156e-01],
        [ 1.5653e-02, -7.9122e-02, -2.3789e-02,  5.3314e-02,  2.7191e-02,
         -9.3753e-02, -4.7187e-02, -7.8702e-02,  4.5353e-02, -4.3735e-02],
        [ 1.7239e-02, -2.2904e-02,  7.7517e-03,  5.6146e-02,  3.6514e-02,
         -1.2995e-01, -6.2474e-0

tensor([[-4.2305e-02, -6.8194e-02,  2.9983e-03, -7.3250e-02,  1.9374e-02,
         -1.8312e-02, -1.8034e-03, -5.2796e-02,  7.1366e-02,  3.7342e-03],
        [-5.6760e-02, -9.1036e-02, -1.4361e-02, -8.4231e-02, -4.6591e-03,
         -4.1311e-02, -6.6963e-03,  6.2170e-02,  1.1670e-01,  3.5783e-02],
        [-2.9188e-02, -1.2092e-01,  1.6318e-02, -5.1330e-02,  4.0309e-02,
         -4.6119e-02, -5.8998e-03,  4.5005e-02,  1.1445e-01,  2.3781e-02],
        [-1.1046e-02, -1.7455e-01,  5.7677e-02, -1.1738e-01,  1.3259e-01,
         -1.7485e-01,  4.0900e-02, -2.4315e-01,  8.7956e-02, -3.7067e-02],
        [-3.2827e-02, -6.7900e-02, -7.6777e-03, -4.9050e-02,  2.7861e-02,
         -1.8763e-02,  1.3931e-02,  2.0249e-02,  5.7402e-02, -2.0339e-02],
        [-7.9606e-02, -1.2465e-01, -3.9206e-03, -1.0322e-01,  6.5180e-02,
         -1.0755e-01, -8.1542e-02, -2.2861e-02,  5.5592e-02,  1.9570e-01],
        [ 4.1339e-02, -3.0752e-02,  4.3311e-02,  6.8260e-02,  7.1750e-02,
         -1.8956e-01, -5.9923e-0

...

tensor([[-0.1466, -1.3119,  1.4614, -0.8578,  1.3958, -0.7632,  1.0294, -1.4254,
          0.6027, -1.0346],
        [-1.5594, -1.5388, -0.8084, -1.5130, -0.9375,  1.5493, -0.8641,  2.0741,
          1.1380,  2.1096],
        [-2.0906, -2.7922, -0.7341, -2.5367, -0.8959,  1.8522, -1.0107,  2.0097,
          1.8937,  3.5179],
        [-2.1836, -2.0878, -0.8995, -1.9510, -1.1547,  2.1510, -1.1161,  3.2837,
          1.6127,  2.0370],
        [-1.4048, -1.5516, -0.2949, -1.4130, -0.4554,  1.2929, -0.5063,  1.6488,
          1.2723,  0.9827],
        [-1.9877, -1.7308, -0.8616, -1.7180, -1.0463,  1.9193, -1.0095,  3.1711,
          1.3404,  1.7089],
        [ 1.2420,  2.6799, -0.1519,  2.3143,  0.0886, -1.6762,  0.2106, -2.0601,
         -1.5670, -1.9904],
        [ 2.4438,  2.0520,  0.7078,  2.5691,  1.0891, -2.9709,  1.3198, -3.8169,
         -1.7511, -3.2686],
        [-2.0871, -3.0875,  0.1206, -2.6066, -0.0810,  1.3653, -0.3237,  1.5999,
          2.7416,  1.3769],
        [-1.0962, -

tensor([[ 8.0075e-01,  2.2265e+00, -6.9972e-01,  2.1928e+00, -3.4706e-01,
         -1.0856e+00, -1.9172e-01, -1.2682e+00, -1.0589e+00, -1.2720e+00],
        [-7.2558e-01, -2.0784e+00,  1.2148e+00, -1.7008e+00,  8.2127e-01,
          1.8727e-01,  6.4960e-01, -3.8748e-01,  1.2230e+00, -1.6305e-01],
        [ 5.8584e-01, -1.1488e+00,  1.5642e+00, -2.4335e-01,  1.6921e+00,
         -1.4915e+00,  1.4705e+00, -2.5305e+00,  3.7363e-01, -1.7853e+00],
        [ 2.7483e+00,  1.0882e+00,  7.6824e-01,  1.7516e+00,  8.9305e-01,
         -2.5466e+00,  1.4459e+00, -3.5326e+00, -1.4154e+00, -2.6655e+00],
        [-2.6890e+00, -3.1796e+00, -1.1951e+00, -2.8819e+00, -1.3621e+00,
          2.5657e+00, -1.3943e+00,  3.0742e+00,  2.1756e+00,  4.1474e+00],
        [ 1.5071e+00,  1.7462e+00, -3.5903e-01,  2.3517e+00,  1.0897e-01,
         -1.7623e+00,  4.2583e-01, -2.3202e+00, -8.8118e-01, -1.9054e+00],
        [ 1.5361e+00,  2.9283e+00, -5.3082e-01,  3.1128e+00,  1.4052e-05,
         -2.0660e+00,  1.5602e-0

tensor([[-1.3019, -1.7888, -0.1368, -1.5285, -0.2394,  1.1762, -0.3305,  0.9349,
          1.4009,  1.2411],
        [-2.0397, -1.8276, -0.8716, -1.7416, -1.0357,  2.0001, -0.9971,  3.1652,
          1.5282,  1.5519],
        [-2.1103, -2.1841, -0.9662, -1.9333, -1.1132,  2.2054, -1.0912,  3.3262,
          1.5218,  2.1491],
        [ 1.0645, -2.1040,  2.0582, -0.7929,  1.9915, -1.8004,  2.0126, -3.4279,
          0.8758, -1.9855],
        [-1.1786, -1.0923, -0.4589, -1.0801, -0.5452,  1.2890, -0.5177,  1.7285,
          0.8109,  0.9103],
        [-2.6015, -3.2472, -1.1581, -2.9816, -1.3612,  2.5489, -1.4399,  2.8093,
          2.0087,  4.6052],
        [ 2.4028,  2.5019,  0.2798,  3.1295,  0.8770, -2.9797,  1.0704, -3.7578,
         -1.9202, -3.1875],
        [-1.4753, -1.4776, -0.5563, -1.4068, -0.7049,  1.5493, -0.7227,  1.9202,
          1.1328,  1.3864],
        [-1.2585, -2.0028, -0.5773, -1.6525, -0.6299,  1.1789, -0.6025,  0.9077,
          1.1286,  2.7555],
        [ 1.9764,  

tensor([[-1.3136e+00, -1.5936e+00, -5.2856e-01, -1.4590e+00, -6.9918e-01,
          1.4551e+00, -6.6297e-01,  1.2999e+00,  1.1072e+00,  1.9132e+00],
        [ 3.5545e+00,  1.2817e+00,  9.9054e-01,  2.0515e+00,  9.6677e-01,
         -3.1571e+00,  1.7509e+00, -4.5421e+00, -1.6816e+00, -3.0588e+00],
        [ 2.7847e+00,  2.2779e+00,  4.8778e-01,  2.7716e+00,  7.8334e-01,
         -2.9856e+00,  1.2399e+00, -4.0586e+00, -1.8034e+00, -3.1639e+00],
        [ 2.1362e+00,  2.8848e+00,  1.5456e-01,  2.6727e+00,  4.4907e-01,
         -2.5116e+00,  7.5973e-01, -3.2527e+00, -1.8802e+00, -2.7543e+00],
        [-2.1760e+00, -3.5807e+00, -8.0064e-01, -2.9499e+00, -1.0442e+00,
          2.0624e+00, -1.1431e+00,  1.5323e+00,  2.0659e+00,  4.7856e+00],
        [ 2.4588e+00,  3.3096e+00, -3.5287e-02,  3.4493e+00,  4.9815e-01,
         -2.9842e+00,  7.4519e-01, -3.7332e+00, -2.2722e+00, -3.0105e+00],
        [ 6.4771e-01, -1.0120e+00,  1.7554e+00, -1.1372e-01,  1.7290e+00,
         -1.6369e+00,  1.5296e+0

tensor([[ 2.5133,  1.6309,  0.5502,  1.9838,  0.6199, -2.2791,  1.1066, -3.1402,
         -1.6992, -2.3551],
        [-2.3075, -2.2719, -0.8316, -2.0469, -1.0802,  2.3355, -1.0845,  3.4881,
          1.8409,  1.5821],
        [ 1.6906, -0.9093,  2.3776,  0.5829,  2.7848, -3.0569,  2.4233, -4.6472,
         -0.0653, -3.4838],
        [-0.0598, -0.8703,  1.0920, -0.5081,  0.9526, -0.4875,  0.6875, -1.1573,
          0.3732, -0.8108],
        [ 1.1397, -1.8605,  3.5463, -0.5537,  3.2818, -2.8718,  2.6275, -4.8085,
          0.2546, -3.1774],
        [-1.7495, -1.7636, -0.5451, -1.6172, -0.7301,  1.7569, -0.7400,  2.2795,
          1.4265,  1.3676],
        [ 0.4887, -2.2063,  3.2140, -1.1063,  3.0287, -2.2282,  2.2694, -3.9644,
          0.6603, -2.4934],
        [ 4.0788,  1.0439,  1.3023,  2.1727,  1.4459, -3.6298,  2.1991, -5.2716,
         -1.7650, -3.6457],
        [-2.4327, -2.6637, -1.1486, -2.3869, -1.3305,  2.6361, -1.3351,  3.2126,
          1.8485,  3.1072],
        [-2.0484, -

In [11]:
torch.save(model, './model.pth')

In [12]:
model = torch.load('model.pth').to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
tensor([[-3.0409e+00, -4.1448e+00, -1.0211e+00, -3.3991e+00, -1.1289e+00,
          2.7220e+00, -1.4238e+00,  2.5203e+00,  2.8514e+00,  4.8063e+00],
        [ 3.6793e+00,  2.9352e-01,  1.1727e+00,  1.5390e+00,  1.2051e+00,
         -3.0194e+00,  2.0353e+00, -4.6530e+00, -1.2751e+00, -2.8644e+00],
        [ 1.1023e+00,  1.6804e+00, -4.1790e-02,  1.6332e+00,  1.7854e-01,
         -1.2655e+00,  3.3381e-01, -1.7099e+00, -1.1206e+00, -1.5388e+00],
        [ 1.9184e+00,  1.3312e+00,  5.6435e-01,  1.6845e+00,  8.0868e-01,
         -2.0304e+00,  1.0674e+00, -2.9210e+00, -1.2030e+00, -2.2968e+00],
        [ 2.1140e+00,  2.9446e+00, -2.7909e-01,  3.3465e+00,  4.0417e-01,
         -2.6128e+00,  6.2761e-01, -3.3895e+00, -1.6943e+00, -2.8750e+00],
        [ 6.6157e-01, -1.8558e+00,  2.6068e+00, -8.2397e-01,  2.3068e+00,
         -1.8303e+00,  1.8973e+00, -3.4455e+00,  6.0563e-01, -2.1089e+00],
        [-2.1950e+00, -2.0939e+00, -9.8639e-01, -1.8590e+00, -1.11

tensor([[ 7.6767e-01,  5.1309e-01,  9.5841e-02,  7.8077e-01,  1.2168e-01,
         -6.1973e-01,  3.3632e-01, -1.0206e+00, -3.6564e-01, -1.0311e+00],
        [-3.0854e-01, -2.9531e+00,  2.7316e+00, -1.8531e+00,  2.5541e+00,
         -1.1968e+00,  1.8870e+00, -2.7046e+00,  1.4321e+00, -1.5310e+00],
        [ 2.0123e-01, -5.8116e-01,  6.1543e-01, -2.0210e-01,  6.5491e-01,
         -3.7566e-01,  6.5760e-01, -9.0229e-01,  1.6334e-01, -7.3677e-01],
        [-2.9170e+00, -4.4722e+00,  2.2291e-01, -3.4840e+00, -1.2171e-02,
          2.0872e+00, -3.8994e-01,  1.9961e+00,  3.7017e+00,  1.9126e+00],
        [ 4.9960e-01, -3.9449e-01,  1.1398e+00,  7.8797e-02,  1.2845e+00,
         -1.0843e+00,  1.0903e+00, -1.9301e+00, -5.5504e-02, -1.5113e+00],
        [-2.6555e+00, -2.4081e+00, -1.1474e+00, -2.1600e+00, -1.3422e+00,
          2.7536e+00, -1.3692e+00,  3.9133e+00,  1.8674e+00,  2.2176e+00],
        [-2.1076e-03, -1.5649e+00,  1.1437e+00, -8.4661e-01,  9.6887e-01,
         -5.1231e-01,  9.5538e-0

tensor([[-6.1442e-01, -5.3837e-01, -1.4039e-01, -4.4748e-01, -2.2975e-01,
          7.0748e-01, -1.9989e-01,  8.4446e-01,  5.0061e-01,  1.3164e-03],
        [-1.5139e+00, -2.2785e+00, -1.5380e-01, -1.8831e+00, -3.1427e-01,
          1.4371e+00, -4.1644e-01,  1.1143e+00,  1.5498e+00,  1.6992e+00],
        [ 1.4087e+00,  2.1699e+00, -6.8376e-01,  2.7279e+00,  2.9069e-02,
         -1.8616e+00,  3.5427e-01, -2.3793e+00, -1.1721e+00, -1.8610e+00],
        [ 4.3363e-01, -6.9350e-01,  1.0744e+00, -9.6535e-02,  1.1371e+00,
         -9.9074e-01,  1.1075e+00, -1.8487e+00,  2.5194e-01, -1.4287e+00],
        [-2.4582e+00, -3.9812e+00,  5.2452e-01, -2.7749e+00,  5.0974e-01,
          1.4206e+00,  8.6982e-02,  8.9927e-01,  3.3446e+00,  1.0675e+00],
        [-2.2919e+00, -2.2533e+00, -1.2774e+00, -2.0530e+00, -1.4188e+00,
          2.6591e+00, -1.3380e+00,  3.7405e+00,  1.3906e+00,  2.6762e+00],
        [-2.3339e+00, -3.4603e+00, -1.0256e+00, -2.6111e+00, -1.0081e+00,
          2.1348e+00, -1.0882e+0

tensor([[-0.1206, -1.7555,  1.9821, -0.9727,  1.8820, -0.9348,  1.4380, -2.0939,
          0.8017, -1.4681],
        [-2.0595, -1.9863, -1.1424, -1.8130, -1.2400,  2.2940, -1.1802,  2.7268,
          1.3686,  2.6452],
        [-2.6572, -3.6921, -1.0484, -3.0696, -1.2212,  2.7936, -1.3313,  2.5210,
          2.3106,  4.5027],
        [-2.9365, -2.7062, -1.3277, -2.3493, -1.5356,  3.1590, -1.5574,  4.4108,
          1.9839,  2.5142],
        [-1.8599, -2.0489, -0.4208, -1.7166, -0.5672,  1.8745, -0.6737,  2.1401,
          1.6192,  1.1553],
        [-2.6852, -2.2054, -1.2657, -2.0391, -1.3946,  2.8249, -1.4173,  4.2733,
          1.6331,  2.0592],
        [ 1.4395,  3.4822, -0.2752,  2.9606,  0.1696, -2.0659,  0.2801, -2.6066,
         -2.0291, -2.3925],
        [ 3.0713,  2.4413,  0.8847,  3.3009,  1.4807, -3.7201,  1.8141, -5.0083,
         -2.1274, -4.0337],
        [-2.6120, -4.1464,  0.1873, -3.1039, -0.0150,  1.9937, -0.3246,  1.8203,
          3.5795,  1.4459],
        [-1.2347, -

tensor([[ 0.7971,  2.7165, -1.0155,  2.7498, -0.4125, -1.2040, -0.2843, -1.4458,
         -1.2889, -1.3932],
        [-0.7714, -2.6319,  1.6352, -1.9595,  1.0926,  0.2786,  0.9273, -0.7710,
          1.4950, -0.4059],
        [ 0.7536, -1.6038,  2.0453, -0.2086,  2.1980, -1.7956,  1.9636, -3.4023,
          0.5763, -2.2742],
        [ 3.5539,  1.1301,  0.9874,  2.1792,  1.1329, -3.1469,  1.9743, -4.6116,
         -1.6788, -3.2089],
        [-3.3792, -4.0666, -1.6534, -3.3939, -1.7774,  3.6461, -1.8271,  3.8287,
          2.5821,  5.2213],
        [ 1.7450,  1.9023, -0.5585,  2.9138,  0.1767, -2.0361,  0.5754, -2.8552,
         -0.9365, -2.1687],
        [ 1.6896,  3.4691, -0.8172,  3.8536,  0.0554, -2.4024,  0.1938, -2.9863,
         -2.0375, -2.3118],
        [ 3.4409,  1.1292,  1.0560,  2.6463,  1.6723, -3.7104,  2.2426, -5.2692,
         -1.5096, -3.8493],
        [ 1.3328,  3.5954, -0.4043,  2.7164,  0.0552, -1.8711,  0.2388, -2.4245,
         -1.9698, -2.2471],
        [-1.3790, -

tensor([[ 1.4710,  3.5917, -0.1641,  2.7301,  0.3235, -2.0781,  0.4201, -2.7094,
         -1.9858, -2.5786],
        [-3.4076, -5.7555,  0.1746, -4.1942, -0.0694,  2.7839, -0.4449,  2.4861,
          4.6395,  2.1996],
        [-0.1989, -1.4334,  1.2839, -0.9411,  1.1154, -0.2861,  0.8981, -1.1340,
          0.8034, -0.8448],
        [-1.9676, -1.7656, -0.8999, -1.4580, -0.9810,  2.1276, -0.9732,  3.2413,
          1.3625,  1.1748],
        [ 1.0398, -2.1039,  2.8138, -0.1973,  3.1157, -2.5957,  2.6591, -4.6913,
          0.8101, -3.0558],
        [ 2.0724,  2.8728, -0.5519,  3.8081,  0.4024, -2.7199,  0.6581, -3.5473,
         -1.6215, -2.8939],
        [ 0.3726, -3.4404,  3.9432, -1.6787,  3.5703, -2.3607,  2.8615, -4.7301,
          1.4787, -2.8550],
        [ 1.5268,  3.9479, -0.2665,  2.8843,  0.2714, -2.1610,  0.4018, -2.8465,
         -2.1005, -2.6969],
        [ 1.0020, -3.0500,  2.9658, -0.9315,  2.9707, -2.2409,  2.7766, -4.5922,
          1.3848, -2.6463],
        [-0.8044, -